In [ ]:
import numpy as np

# Vector spaces and bases

There are many other "spaces" of objects for which the same concepts of linear combinations, spans, linear transformations, linear dependence, etc. from $n$-space continue to hold, provided we appropriately define "vector addition" and "vector scaling" (a.k.a. "scalar multiplication (of vectors)").

## Real-valued functions

We can represent a function $f \colon \mathbb{R} \to \mathbb{R}$ using a Python function whose input is a floating point number and whose output is a floating point number.

(This is not a perfect representation, because not every real number can be represented as a floating point number, but this is not too bad.)

In [ ]:
# vector addition
def add(u, v):
    def w(x):
        return u(x) + v(x)
    return w

# scalar multiplication
def scale(c, v):
    def w(x):
        return c * v(x)
    return w

In [ ]:
# your favorite real numbers at which to evaluate functions
x = 0.
y = np.pi/4.
z = np.pi/2.

In [ ]:
# add the cosine and exponential functions
f = add(np.cos, np.exp)

In [ ]:
f(x)

2.0

In [ ]:
np.cos(x) + np.exp(x)

2.0

In [ ]:
# this should give the additive inverse of f
g = scale(-1., f) 

In [ ]:
g(x)

-2.0

In [ ]:
g(x) + np.exp(x)

-1.0

In [ ]:
# this should give the "zero function"
zero_func = add(f, g)

In [ ]:
zero_func(x)

0.0

In [ ]:
zero_func(y)

0.0

In [ ]:
zero_func(z)

0.0

Checking for the equality of two function $f \colon \mathbb{R} \to \mathbb{R}$ and $g \colon \mathbb{R} \to \mathbb{R}$ is difficult, because there are infinitely-many (indeed, "uncountably-many") $x$'s at which $f$ and $g$ must agree.

(We represent $f$ and $g$ using Python functions that only allow floating point numbers as inputs. Since there are finitely-many floating point numbers, we could, in principle, check if $f(x) = g(x)$ for all floating point numbers $x$. But this would take a very long time.)

This makes it difficult to check if a function can be written as a linear combination of other functions.

## Polynomials

Numpy implements a Polynomial class that supports vector space operations on polynomials $\mathsf{P}(\mathbb{R})$.

In [ ]:
from numpy.polynomial import Polynomial as P

In [ ]:
# create a polynomial by specifying coefficients in a list
p = P([1,2,3])
print(p)

1.0 + 2.0·x¹ + 3.0·x²


In [ ]:
p

Polynomial([1., 2., 3.], domain=[-1,  1], window=[-1,  1])

In [ ]:
# the degree of p
p.degree()

2

In [ ]:
# evaluate the polynomial at some real numbers
p(0.), p(1.), p(2.)

(1.0, 6.0, 17.0)

In [ ]:
q = P([1,0,0,0,0,0,7])
print(q)

1.0 + 0.0·x¹ + 0.0·x² + 0.0·x³ + 0.0·x⁴ + 0.0·x⁵ + 7.0·x⁶


In [ ]:
q

Polynomial([1., 0., 0., 0., 0., 0., 7.], domain=[-1,  1], window=[-1,  1])

In [ ]:
q.degree()

6

In [ ]:
# numpy implements some other nice things for polynomials, e.g., derivatives
print(q.deriv())

0.0 + 0.0·x¹ + 0.0·x² + 0.0·x³ + 0.0·x⁴ + 42.0·x⁵


In [ ]:
q.deriv()

Polynomial([ 0.,  0.,  0.,  0.,  0., 42.], domain=[-1.,  1.], window=[-1.,  1.])

In [ ]:
# the numpy polynomial objects allow you to add and scale polynomials using the infix operators + and *
def add(p, q):
    return p + q

def scale(c, p):
    return c * p

In [ ]:
r = add(p,q)
print(r)

2.0 + 2.0·x¹ + 3.0·x² + 0.0·x³ + 0.0·x⁴ + 0.0·x⁵ + 7.0·x⁶


In [ ]:
r

Polynomial([2., 2., 3., 0., 0., 0., 7.], domain=[-1.,  1.], window=[-1.,  1.])

In [ ]:
# this should be the additive inverse of p
minus_p = scale(-1., p)
print(minus_p)

-1.0 - 2.0·x¹ - 3.0·x²


In [ ]:
minus_p

Polynomial([-1., -2., -3.], domain=[-1.,  1.], window=[-1.,  1.])

In [ ]:
# this should give the zero polynomial
zero_poly = add(p, minus_p)
print(zero_poly)

0.0


In [ ]:
zero_poly

Polynomial([0.], domain=[-1.,  1.], window=[-1.,  1.])

In [ ]:
zero_poly(0.), zero_poly(1.), zero_poly(2.)

(0.0, 0.0, 0.0)

In [ ]:
# sadly, the implementation of degree is not correct in Numpy version 1.23.2 ☹️; the correct answer is -1.
zero_poly.degree()

0

It is easy to check if two polynomials are equal because each has only finitely-many non-zero coefficients.

In [ ]:
def poly_equal(p, q):
    return add(p, scale(-1., q)) == P([0.])

In [ ]:
# Check if the derivative of 1 + x + x^2 is equal to 1 + 2x
poly_equal(P([1,1,1]).deriv(), P([1,2]))

True

The space $\mathsf{P}_d(\mathbb{R})$ of polynomials with degree at most $d$ is a subspace of the vector space of all polynomials.
An ordered basis for $\mathsf{P}_d(\mathbb{R})$ is $\mathcal{B}_d = (1, x, x^2, \dotsc, x^d)$.

Numpy makes it easy to obtain the coordinate representation of a polynomial $p$ with respect to the ordered basis $\mathcal{B}_d$ for $d \geq \operatorname{deg}(p)$.

In [ ]:
# coordinate representation of polynomial with respect to $\mathcal{B}_d$
def rep_poly(p, d = None):
    pdeg = p.degree()
    if d == None:
        d = pdeg
    if pdeg <= d:
        return np.concatenate((p.coef, np.zeros(d-pdeg)))
    elif d >= -1 and pdeg == 0 and p.coef[0] == 0.:
        # this handles the bug about degree of the zero polynomial
        return np.zeros(d+1)
    else:
        raise Exception('Degree of p ({0}) is larger than d ({1})'.format(pdeg, d))

In [ ]:
print(p)

1.0 + 2.0·x¹ + 3.0·x²


In [ ]:
rep_poly(p, 6)

array([1., 2., 3., 0., 0., 0., 0.])

In [ ]:
print(q)

1.0 + 0.0·x¹ + 0.0·x² + 0.0·x³ + 0.0·x⁴ + 0.0·x⁵ + 7.0·x⁶


In [ ]:
rep_poly(q, 6)

array([1., 0., 0., 0., 0., 0., 7.])

In [ ]:
print(zero_poly)

0.0


In [ ]:
rep_poly(zero_poly, -1) # this is correct

array([], dtype=float64)

In [ ]:
rep_poly(zero_poly, 0) # also correct

array([0.])

In [ ]:
rep_poly(zero_poly, 6) # yup

array([0., 0., 0., 0., 0., 0., 0.])

We can obtain the polynomial whose coordinate representation is equal to the sum of the coordinate representations of $p$ and $q$ ...

In [ ]:
p_plus_q = P(rep_poly(p, 6) + rep_poly(q, 6))
print(p_plus_q)

2.0 + 2.0·x¹ + 3.0·x² + 0.0·x³ + 0.0·x⁴ + 0.0·x⁵ + 7.0·x⁶


In [ ]:
p_plus_q

Polynomial([2., 2., 3., 0., 0., 0., 7.], domain=[-1,  1], window=[-1,  1])

... this should be the same as $p + q$

In [ ]:
poly_equal(p_plus_q, add(p,q))

True

In [ ]:
zero_poly.coef[0] == 0.

True